In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch
import numpy as np
import urllib.request
import sys, os
import glob
from PIL import Image

# (선택) decord로 비디오 읽기 – 안 쓰고 직접 frame 텐서를 만들고 싶으면 이 부분만 바꿔도 됨
from decord import VideoReader, cpu

In [7]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model import Model

C:\Users\JM\anaconda3\envs\t2v\lib\site-packages\accelerate\utils\torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\JM\PycharmProjects\Text2Video-Zero-main\annotator\openpose\body.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
C:\Users\JM\PycharmProjects\Text2Video-Zero-main\annotator\openpose\hand.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
C:\Users\JM\anaconda3\envs\t2v\lib\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a dep

cuda
cuda


In [8]:
# -----------------------------------
# 1. device 설정 & 모델 / preprocessor 로드
# -----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# torch.hub에서 preprocessor / encoder 로드
processor = torch.hub.load('facebookresearch/vjepa2', 'vjepa2_preprocessor')
loaded  = torch.hub.load('facebookresearch/vjepa2', 'vjepa2_vit_giant')

if isinstance(loaded, tuple):
    vjepa2_encoder = loaded[0]    # encoder만 사용
else:
    vjepa2_encoder = loaded

vjepa2_encoder = vjepa2_encoder.to(device).eval()

Using cache found in C:\Users\JM/.cache\torch\hub\facebookresearch_vjepa2_main
Using cache found in C:\Users\JM/.cache\torch\hub\facebookresearch_vjepa2_main


In [21]:
# ---------------------------------------------------
# 2. frames 폴더에서 _0.jpg ~ _7.jpg 읽기
# ---------------------------------------------------
frames_dir = "../frames"

# 파일 목록 정렬 (_0.jpg, _1.jpg, ... 순서 보장)
frame_paths = glob.glob(os.path.join(frames_dir, "*.jpg"))

def frame_key(path):
    # 예: path = "frames/_3.jpg" -> stem = "_3" -> idx = 3
    name = os.path.splitext(os.path.basename(path))[0]  # "_3"
    idx = int(name.split("_")[-1])
    return idx

frame_paths = sorted(frame_paths, key=frame_key)

print("frames:", frame_paths)

# 각 프레임을 np.ndarray(H,W,3)로 읽어서 리스트로 만들기
imgs = [Image.open(p).convert("RGB") for p in frame_paths]
buffer = [np.array(im) for im in imgs]   # <-- ★ 여기! 리스트로 전달할 애

frames: ['../frames\\_0.jpg', '../frames\\_1.jpg', '../frames\\_2.jpg', '../frames\\_3.jpg', '../frames\\_4.jpg', '../frames\\_5.jpg', '../frames\\_6.jpg', '../frames\\_7.jpg']


In [24]:
# ---------------------------------------------------
# 2) processor + encoder
# ---------------------------------------------------
# processor는 "프레임 리스트"를 기대함
inputs = processor(buffer)   # 여기서 VideoTransform.__call__이 돌아감

# 보통 [clip_tensor] 형태로 나옴
if isinstance(inputs, list):
    clip = inputs[0]
else:
    clip = inputs

print("clip type:", type(clip))
if torch.is_tensor(clip):
    print("clip shape:", clip.shape)  # 기대: (T, 3, H, W) 또는 (3, H, W, T 등)
else:
    raise TypeError(f"Unexpected clip type from processor: {type(clip)}")

# 만약 shape가 (T, H, W, 3)이면, 아래처럼 바꿀 수도 있음:
if clip.ndim == 4 and clip.shape[-1] == 3:
    # (T, H, W, 3) -> (T, 3, H, W)
    clip = clip.permute(0, 3, 1, 2)

# 이제 encoder에 넣을 입력 텐서
video_tensor = clip.unsqueeze(0).to(device)   # torch.Size([1, 3, 8, 256, 256])

with torch.no_grad():
    feats = vjepa2_encoder(video_tensor)

print("feats type:", type(feats))
if torch.is_tensor(feats):
    print("feats shape:", feats.shape)
else:
    for k, v in feats.items():
        if torch.is_tensor(v):
            print("feat:", k, v.shape)

clip type: <class 'torch.Tensor'>
clip shape: torch.Size([3, 8, 256, 256])


C:\Users\JM\anaconda3\envs\t2v\lib\contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


feats type: <class 'torch.Tensor'>
feats shape: torch.Size([1, 1024, 1408])
